# Generate Unrelaxed Surfaces
Use [CatKit](https://catkit.readthedocs.io/en/latest/) to produce initial slab structures and enumerate the adsorbate sites.

In [1]:
from catkit.gen.adsorption import AdsorptionSites
from catkit.gen.surface import SlabGenerator
from pathlib import Path
from ase.io import read
from examol.utils.conversions import read_from_string, write_to_string
import json

Configuration

In [2]:
bulk_path: str = './relaxed-bulk/fcc-Pd.extxyz'  # Path to a relaxed bulk structure
facet: tuple[int, int, int] = (1, 1, 1)  # Which surface to use
supercell: tuple[int, int] = (3, 3)  # Size of the supercell
layers: int = 6  # Number of layers for the slab

Derived

In [3]:
bulk_path = Path(bulk_path)
bulk_name = bulk_path.name.split(".")[0]
output_name = f'{bulk_name}_{"".join(map(str, facet))}_{"x".join(map(str, supercell))}'
out_dir = Path('surfaces') / output_name
out_dir.mkdir(parents=True)

## Load in the Structure
Into an ase.Atoms object

In [4]:
atoms = read(bulk_path)
atoms

Atoms(symbols='Pd', pbc=True, cell=[[1.0167062034150769e-07, 1.9307360191576008, 1.9307356679659875], [1.930735554740657, 1.0167063715463548e-07, 1.9307356679659706], [1.9307355547405736, 1.9307360191575014, 1.0167072120474846e-07]], forces=..., calculator=SinglePointCalculator(...))

## Make the Surface Generator
CatKit uses a [SlabGenerator](https://catkit-jboes.readthedocs.io/en/latest/_static/frontmatter/catgen.html) object to produce surfaces and detect adsorption sites. 
It takes the desired surface geometry as inputs

In [5]:
gen = SlabGenerator(
    atoms,
    miller_index=facet,
    layers=layers,
    fixed=layers // 2,  # Fix the bottom half
    layer_type='trim',  # Get the desired number of layers
    standardize_bulk=True,
    vacuum=10,  # Will expand the vacuum layer
    tol=1e-20,
)

## Save each slab
Save the unrelaxed slab and coordinates for the absorption sites

In [6]:
terminations = gen.get_unique_terminations()
print(f'Found {len(terminations)} unique terminations')

Found 1 unique terminations


Iterate over the terminations

In [7]:
for term in range(len(terminations)):
    # Create the slab then detect the adsorption sites
    slab = gen.get_slab(size=supercell, iterm=term)
    sites = AdsorptionSites(slab)
    
    # Get both the coordinates and vectors for each sites
    coords = sites.get_coordinates()
    vectors = sites.get_adsorption_vectors()
    conn = sites.get_connectivity()
    
    # Save the slab and site information
    slab_dir = out_dir / f'term={term}'
    slab_dir.mkdir()
    
    slab.write(slab_dir / 'unrelaxed.extxyz', columns=['symbols', 'positions', 'move_mask'])
    with slab_dir.joinpath('site-info.json').open('w') as fp:
        json.dump({
            'coords': coords.tolist(),
            'vectors': vectors.tolist(),
            'connectivity': conn.tolist(),
        }, fp)